In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_3') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(random_state=107,
                                        max_depth=10,
                                        min_samples_leaf=2,
                                        min_samples_split=5,
                                        n_estimators=50),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,15,30.0,30.0
1,HOME_&_GARDEN_1_027,107,183.0,213.0
2,HOME_&_GARDEN_1_053,247,230.0,205.0
3,HOME_&_GARDEN_1_140,106,107.0,102.0
4,HOME_&_GARDEN_1_177,128,127.0,124.0
...,...,...,...,...
95,SUPERMARKET_3_499,1042,250.0,230.0
96,ACCESORIES_1_158,129,114.0,106.0
97,SUPERMARKET_3_282,1366,1543.0,1387.0
98,ACCESORIES_1_354,174,190.0,188.0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

38574
41117.0
-2543.0


In [4]:
tienda =  pd.DataFrame(resultados)
file_name = "NYC_3_ventas.xlsx"
tienda.to_excel(file_name)

In [4]:
pd.DataFrame(resultados) #7

,producto,prediccion,test
0,ACCESORIES_1_108,"[1.2018967212479286, 0.9317060953735924, 0.680...","[0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, ..."
1,HOME_&_GARDEN_1_027,"[4.579438005604671, 5.631064183838974, 3.83296...","[4, 2, 4, 4, 4, 2, 5, 0, 7, 3, 3, 4, 5, 5, 6, ..."
2,HOME_&_GARDEN_1_053,"[8.07279360797037, 12.249357142857143, 7.50251...","[8, 11, 7, 4, 4, 14, 7, 5, 20, 10, 11, 4, 10, ..."
3,HOME_&_GARDEN_1_140,"[4.695080000456212, 4.894346251675097, 3.55030...","[1, 3, 1, 0, 5, 2, 4, 5, 7, 2, 0, 1, 6, 9, 8, ..."
4,HOME_&_GARDEN_1_177,"[5.265479624283134, 5.508532262551524, 3.90288...","[8, 7, 0, 8, 3, 4, 1, 17, 25, 30, 18, 6, 0, 0,..."
...,...,...,...
95,SUPERMARKET_3_499,"[11.351199086671045, 7.760995115995116, 13.383...","[0, 0, 14, 16, 28, 23, 22, 51, 39, 65, 41, 40,..."
96,ACCESORIES_1_158,"[5.4059805680797695, 6.203440681627569, 4.8017...","[5, 4, 5, 2, 7, 5, 10, 6, 1, 8, 4, 3, 8, 9, 5,..."
97,SUPERMARKET_3_282,"[58.154491303724484, 79.46875396825395, 64.333...","[77, 104, 53, 45, 35, 22, 34, 63, 85, 55, 46, ..."
98,ACCESORIES_1_354,"[4.387938418079296, 8.684144869502479, 6.32579...","[7, 5, 2, 5, 2, 7, 5, 2, 5, 7, 6, 2, 4, 1, 5, ..."


In [7]:
pd.DataFrame(resultados) #14

,producto,prediccion,test
0,ACCESORIES_1_108,"[1.1669603174603176, 0.6981865079365077, 0.821...","[0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, ..."
1,HOME_&_GARDEN_1_027,"[4.638481240981241, 4.99316341991342, 4.564210...","[4, 2, 4, 4, 4, 2, 5, 0, 7, 3, 3, 4, 5, 5, 6, ..."
2,HOME_&_GARDEN_1_053,"[7.447278499278497, 8.355496753246754, 7.78650...","[8, 11, 7, 4, 4, 14, 7, 5, 20, 10, 11, 4, 10, ..."
3,HOME_&_GARDEN_1_140,"[4.387232212232214, 5.42890873015873, 4.776258...","[1, 3, 1, 0, 5, 2, 4, 5, 7, 2, 0, 1, 6, 9, 8, ..."
4,HOME_&_GARDEN_1_177,"[5.214388888888888, 5.885666666666666, 5.07606...","[8, 7, 0, 8, 3, 4, 1, 17, 25, 30, 18, 6, 0, 0,..."
...,...,...,...
95,SUPERMARKET_3_499,"[8.44593073593074, 9.046761904761905, 12.97389...","[0, 0, 14, 16, 28, 23, 22, 51, 39, 65, 41, 40,..."
96,ACCESORIES_1_158,"[4.739011904761904, 6.031991341991344, 4.72572...","[5, 4, 5, 2, 7, 5, 10, 6, 1, 8, 4, 3, 8, 9, 5,..."
97,SUPERMARKET_3_282,"[60.30745115995115, 80.55711507936506, 59.0159...","[77, 104, 53, 45, 35, 22, 34, 63, 85, 55, 46, ..."
98,ACCESORIES_1_354,"[5.80095634920635, 6.225952380952378, 6.064591...","[7, 5, 2, 5, 2, 7, 5, 2, 5, 7, 6, 2, 4, 1, 5, ..."


## Predigo 30 dias(los de test) 